# Classificador Valência em Tweets
Este Notebook contém os resultados de uma das etapas do Projeto Final da disciplina de "IA369-Y" na UNICAMP. São apresentados aqui os resultados da implementação de um classificador de valência (1 a 7) em tweets utilizando uma abordagem de aprendizado de máquina, especificamente, 

## Dataset 
O conjunto de dados utilizados nesta implementação são referentes aos tweets importados pela API [Tweepy](http://www.tweepy.org/) e pré-processados. O dataset é composto com as seguintes colunas:

1. id - Usuário no Twitter
2. txt - O conteúdo do tweet
3. val - Valência de 1 a 7
4. int - Intensidade de 1 a 7
5. cit - Cidade e País do usuário
6. data - Data e hora do tweet
7. dia - Dia da semana

1318 amostras x 7 colunas

Para o tratamento dos emojis, foram gerados datasets com diferentes tratamentos:

1. **output-pleasure-arousal-labeled**- Os emojis presentes nos tweets são excluídos
2. **output-pleasure-arousal-labeled-emoji** - Os emojis são mantidos, porém sem tratamento, foi observado que em uma das etapas eles são desconsiderados
3. **demojized_emojios** - Os emojis são tratados pela biblioteca [Demojize](https://github.com/nkmrtty/demojize.py/blob/master/demojize.py), ou seja, são transcritos, por exemplo *:smirking_face:* e *:yellow_heart:*

Ao final, o dataset selecionado foi o **demojized_emojios**, pois foi possível incluir os emojis na abordagem adotada pelo classificador. Entendemos que emojis, principalmente nos tweets, carregam informações valiosas sobre a valência e intensidade.

## Linguagens e Bibliotecas

1. Ambiente: [Anaconda3 4.3.1](https://repo.continuum.io/archive/index.html)
2. Linguagem de Programação: [Python 3.3](https://www.python.org/) 
3. Biblioteca de Dataframe: [Panda 0.19.2](http://pandas.pydata.org/).
4. Machine Learning: [Scikit-learn](http://scikit-learn.org/stable/index.html)
5. Plotting: [Matplotlib](https://matplotlib.org/)

## Abordagem
O dataset foi pré-processado, foram extraídas *features* de texto, separados em dados de treinamento e teste, e por fim, implementado um algoritmo de *Supporte Vector Machines* (SVM), conforme detalhes abaixo:

In [ ]:
# encoding: utf-8
# encoding: iso-8859-1
# encoding: win-1252

import pandas as pd
import csv
from datetime import datetime
from dateutil import parser
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns 


## Pré-processamento
As etapas adotadas para pré-processar o dataset foram:

1. Separação de Data e Hora
2. Substituição das Cidades por Valores
3. Criação de Features [É madrugada e É final de semana]
4. Balanceamento das amostras
5. Pré-processamento dos tweets
6. Tokenizing
7. Remoção das StopWords

### Dataset com Demojize

In [ ]:
### Com Demogize
datasetInFrame = pd.read_csv("demojized_emojios.csv", sep="|",quoting=csv.QUOTE_ALL)
### Colocando espaços entre os emojis
sentencesEmojis = [str(sentence).lower().replace(":"," ") for sentence in datasetInFrame["txt"]]
datasetInFrame["txt"] = sentencesEmojis
datasetInFrame.shape

### Dataset com Emoji

In [ ]:
### Com emoji
#datasetInFrame = pd.read_csv("output-pleasure-arousal-labeled-emoji.csv", sep="|",quoting=csv.QUOTE_ALL)

### Dataset sem Emoji

In [ ]:
### Sem emoji
#datasetInFrame = pd.read_csv("output-pleasure-arousal-labeled.csv", sep="|", encoding="ISO-8859-1",quoting=csv.QUOTE_ALL)

In [ ]:
datasetInFrame.head()

### Distribuição das Classes do Dataset
- Valência (val): 1 a 7
- Intensidade (int): 1 a 7

#### Valência

In [ ]:
totalValClasses = {}

for i in range(1,8):
    totalValClasses[i] = datasetInFrame[datasetInFrame.val == i]["txt"].count()
totalValClasses

In [ ]:
sns.countplot(x='val', data=datasetInFrame)

#### Intensidade

In [ ]:
totalIntClasses = {}

for i in range(1,8):
    totalIntClasses[i] = datasetInFrame[datasetInFrame.val == i]["txt"].count()
totalIntClasses

In [ ]:
sns.countplot(x='int', data=datasetInFrame)

### Separação de data e hora
Para gerar *features* posteriormente, foi adotada a estratégia separar os dados de data e hora.

In [ ]:
dateTime = datasetInFrame["data"].apply(lambda  x: x.split(' '))
date = dateTime.apply(lambda x: x[0])
time = dateTime.apply(lambda x: x[1])
datasetInFrame["date"] = date
datasetInFrame["time"] = time
del datasetInFrame["data"]
datasetInFrame.head()

### Substituição de Valores Para Cidade
1. Removendo o País;
2. Substituindo: 
    * Rio de Janeiro = 1 
    * São Paulo = 2

Essa estratégia foi adotada para incluir a cidade como uma *feature* no classificador. Apesar desse processamento, essa coluna não foi utilizada na implementação do modelo apresentado aqui.

In [ ]:
city = datasetInFrame["cit"].apply(lambda x: x.split(','))
datasetInFrame["cit"] = city.apply(lambda x: x[0])
datasetInFrame.head()

In [ ]:
datasetInFrame["cit"] = datasetInFrame["cit"].apply(lambda x: 1 if x == "Rio de Janeiro" else 2)
datasetInFrame.head()

### Criação de  *features*

Após o processamento, definimos como *features* interessantes a serem consideradas como Final de Semana e Madrugada. Intuitivamente, os tweets tendem a ser mais felizes nos finais de semana, mas será que tal característica é de fato relevante? Como uma tentativa de responder a essa questão, foram adicionadas duas colunas de:

1. isWeekend? [dia 6 após às 19:00, dia 7 e 1 até 19:00 - 1 sim, 0 não
2. isMad? [23:00 às 5:00] - 1 sim, 0 não



In [ ]:
def isWeekend(day,hour):
    format = '%H:%M:%S'
    hour = datetime.strptime(hour, format)
    limitWeekend = datetime.strptime("19:00:00", format)
    if (day == 6) and (hour >= limitWeekend):
        return 1
    elif (day == 1) and (hour < limitWeekend):
        return 1
    elif day == 7:
        return 1
    else:
        return 0

datasetInFrame['isWeekend'] = pd.Series(np.zeros(len(datasetInFrame)), index=datasetInFrame.index)

for index, row in datasetInFrame.iterrows():
    datasetInFrame.loc[index,"isWeekend"] = isWeekend(row["dia"],row["time"])
datasetInFrame.head()

In [ ]:
def isMad(hour):
    format = '%H:%M:%S'
    hour = datetime.strptime(hour, format)
    initLimit = datetime.strptime("23:00:00", format)
    endLimit = datetime.strptime("05:00:00", format)
    
    if (hour >= initLimit) or (hour < endLimit):
        return 1
    else:
        return 0

    
datasetInFrame['isMad'] = pd.Series(np.zeros(len(datasetInFrame)), index=datasetInFrame.index)

for index, row in datasetInFrame.iterrows():
    datasetInFrame.loc[index,"isMad"] = isMad(row["time"])
datasetInFrame.head()

### Histogramas do Dataset
Como uma forma de analisar a influência de tais *features* os histogramas abaixo foram gerados.

#### Valência entre Final de Semana (1.0) e Dia de Semana (0.0)
Em termos de números, o dataset possui muito mais tweets classificados em dias semana do que dia de final de semana, entretanto quando dados são convertidos em percentual, apresentam uma sensível diferença. Os tweets em finais de semana tendem a ser mais felizes (sensivelmente)

In [ ]:
#Valência em Fim de Semana
datasetInFrame.hist(column='val',by='isWeekend', bins=10)

#### Valência na Madrugada (1.0) e Não (0.0)

In [ ]:
#Valência na Madrugada
datasetInFrame.hist(column='val',by='isMad', bins=10)

#### Valência por Cidade

In [ ]:
#Valência por Cidade
#datasetInFrame.hist(column='val',by='cit', bins=10)

#### Valência por Intensidade

In [ ]:
#Valência por Intensidade
#datasetInFrame.hist(column='val',by='int', bins=10)

#### Valência por Dia

In [ ]:
#Valência por Dia
#datasetInFrame.hist(column='val',by='date', bins=10)

### Balanceamento  das amostras
Outra etapa importante do pré-processamento é o balanceamento dos dados, ou seja, realizar uma distribuição mais uniforme de amostras para as classes de valência. Para tornar mais balanceado, foram selecionadas até 100 amostras de cada.

In [ ]:
balancedDataset = pd.DataFrame()
limitSamples = 100
for i in datasetInFrame['val'].unique():
    balancedDataset = balancedDataset.append(datasetInFrame[datasetInFrame['val'] == i].iloc[:limitSamples],ignore_index=True)
    
balancedDataset.head()

In [ ]:
sns.countplot(x='val', data=balancedDataset)

In [ ]:
#Seleciona aleatóriamente
dataset = balancedDataset.sample(len(balancedDataset), replace=True)
sns.countplot(x='val', data=dataset)

In [ ]:
dataset.head()

### Pré-processamento dos tweets
Remoção de caracteres especiais e conversão em letras minúsculas.

In [ ]:
sentences = [str(sentence).lower().replace("'","").replace(".","").replace(",","").replace('"',"").replace("?","") for sentence in dataset["txt"]]
dataset["txt"] = sentences
dataset.head()

### Tokenizing

In [ ]:
sentencesWithTokens = [nltk.word_tokenize(sentence.lower()) for sentence in dataset["txt"]]
dataset["txt"] = sentencesWithTokens
dataset.head()

### Remoção das StopWords

In [ ]:
stops = set(stopwords.words("portuguese"))
#words = ' '.join([w for w in words if not w in stops])

sentencesWithoutStopWords = []
for i,row in dataset.iterrows():
    sentence =  ' '.join([w for w in row["txt"] if not w in stops])
    sentencesWithoutStopWords.append(sentence.strip())
dataset["txt"]=sentencesWithoutStopWords
dataset.head()

## *Features* de Texto

Uma comum abordagem de classificação de texto é a utilização de *Bag of Words*, que fará a contagem da ocorrência de uma palavra nos dados de treino, e em seguida, o *Term Frequency-Inverse Document Frequency* que divide o número de ocorrências de cada palavra em um documento pelo número total de palavras no documento. Como estratégia, escolhemos utilizar Bigrams(ngram), que considerará a ocorrência cada palavra e a anterior

### *Bag Of Words*

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(token_pattern=r"(?u)\b[a-zA-Z]\w+\b",lowercase=True, ngram_range=(1, 2))
datasetTrain = count_vect.fit_transform(dataset["txt"])

###Visualização
#from pprint import pprint
#pprint(count_vect.vocabulary_)

In [ ]:
#datasetTrain.todense()
print("Total de features: ", datasetTrain.shape)

### *Term Frequency-Inverse Document Frequency*

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(norm="l2",use_idf=True)
X_tfidf = tfidf_transformer.fit_transform(datasetTrain)

## Separação do dataset
A abordagem definida foi de 70% para treino e 30% para teste

In [ ]:
train = datasetTrain[:int(datasetTrain.shape[0] * 0.7)]
trainTarget = dataset['val'][:int(len(dataset['val']) * 0.7)]

test = datasetTrain[:-int(datasetTrain.shape[0] * 0.3)]
testTarget = dataset['val'][:-int(len(dataset['val']) * 0.3)]


## Abordagem com SVM
A abordagem com o SVM foi escolhida, pois ele apresenta bons resultados e por experiências anteriores com o algoritmo.

A implementação utilizada disponível em [scikit-learn](http://scikit-learn.org/stable/index.html) uma biblioteca em Python amplamente utilizada para tarefas de *Machine Learning*.

Parâmetros:
* Kernel = 'rbf', recomendado por algumas referências [1] como uma boa escolha de primeira abordagem. O "poly" teve um desempenho muito ruim e o "linear" apresentou um score alto.
* Gamma = 0, apresentou bons resultados quando "rbf"
* C = 30, apresentou bons resultados quando "rbf", quando 100 aproximou de 98%


In [ ]:
#svm = SVC(kernel="linear") # ~ 99 
#svm = SVC(kernel="poly") # ~ 19 
#svm = SVC(kernel='rbf',gamma=0.001, C=100.) # ~ 98
svm = SVC(kernel='rbf',gamma=0.001, C=30.) # ~87

In [ ]:
print("Original number of features : %d" % train.shape[1])
svm.fit(train, trainTarget)
predict = svm.predict(test)
print("Score: ",svm.score(test, testTarget))

### Matriz de Confusão

In [ ]:
print (pd.crosstab(testTarget, predict, rownames=['Real'], colnames=['Classificado'], margins=True))
#print(confusion_matrix(testTarget, predict, labels = [1, 2, 3,4,5,6,7]))

In [ ]:
balancedDataset.head()

## Validação

Para validar o classificador foi utilizado um dataset sem labels, o dataset é composto das seguintes colunas:

1. id - Usuário no Twitter
2. txt - O conteúdo do tweet
3. cit - Cidade e País do usuário
4. data - Data e hora do tweet
5. dia - Dia da semana

Total: 4202 amostras x 5 colunas.

O dataset passou pelo mesmo pré-processamento de emoji que o dataset utilizado para treino.

In [ ]:
datasetValida = pd.read_csv("demojized_input-emoji.csv", sep="|",quoting=csv.QUOTE_ALL)

### Colocando espaços entre os emojis
sentencesEmojis = [str(sentence).lower().replace(":"," ") for sentence in datasetValida["txt"]]
datasetValida["txt"] = sentencesEmojis

In [ ]:
tweets = datasetValida["txt"]
validaCounts = count_vect.transform(tweets)
validaTfidf = tfidf_transformer.transform(validaCounts)

validaPredict = svm.predict(validaTfidf)

for doc, category in zip(tweets, validaPredict):
    #print(doc, " ", category)
    pass

### Valência e Intensidade
A valência foi predita pelo modelo e a intensidade adotada como padrão foi a 4 (neutra).

In [ ]:
datasetValida["val"] = pd.Series(validaPredict, index=datasetValida.index)

#Todas as intensidades como 4
intensity = [4]*len(datasetValida)
datasetValida["int"] = pd.Series(intensity, index=datasetValida.index)

datasetValida.head()

### Tratamento de dados para saída

In [ ]:
datasetValida["txt"] = sentencesEmojis
datasetValida.head()

formato = '%d/%m/%Y %H:%M'
dateTimeConverted = [datetime.strptime(dt, formato).strftime('%m/%d/%y %H:%M') for dt in datasetValida["data"]]
datasetValida["data"] = dateTimeConverted
sentencesValida = [str(sent) for sent in datasetValida["txt"]]
datasetValida["txt"] = sentencesValida

### Histograma das classificações

In [ ]:
sns.countplot(x='val', data=datasetValida)

In [ ]:
datasetValida.head()

In [ ]:
datasetValida.to_csv('output-classificador-com-demoji-rbf-85.csv', ",", index=False)
print("Arquivo gerado: output-classificador-com-dmoji-85.csv ")

## Conclusões
Foi possível obter uma classificação inicial válida e com bons resultados (em muitos casos "overfitting") a partir do SVM. Avaliando os diferentes parâmetros selecionados, obtivemos os seguintes resultados:

* Kernel Linear - 99% e se apresentando um classificador "otimista"
* Kernel Polynomial - 19% 
* Kernel Radial Basis Function (gamma=0.001, C=100) - 98% e se apresentando um classificador "pessimista"
* Kernel Radial Basis Function (gamma=0.001, c=30) - 87% e classificou apenas as classes 3,4 e 5 (também "pessimista")

Na matriz de confusão do Kernel Radial Basis Function (gamma=0.001, c=30), o modelo classificou muito bem as classes, errando mais na classe de valência 3, indicando uma forte tendência em classificar os tweets como negativos, o que chamamos aqui de “pessimista”. Outro interessante comportamento do classificador com tais parâmetros, é que classificou apenas as classes 3,4 e 5, o que podemos trocar por Negativo, Neutro e Positivo.

Apesar das classificações serem dadas como "pessimistas" no caso do RBF, as classificações se mostraram, de certa forma, coerentes com o conteúdo dos tweets.

Entre as dificuldades e fraquezas, podem ser citadas o processo de extração de *features*, nesse caso, para se aproximar do objetivo da ferramenta proposta no projeto, devemos estudar e adicionar outras *features* a serem consideradas no classificador, tais como final de semana, madrugada, entre outras que julgarmos relacionadas. 
No decorrer da implementação, encontramos algumas dificuldades em considerar os emojis, mas ao final da implementação, eles foram adicionados ao BoW e ao classificador.

Uma questão importante a ser considerada, é de que os tweets possuem muitas gírias, siglas e erros ortográficos. Dessa forma, acreditamos que, como próximos passos, seja uma avaliação mais profunda no impacto disso no classificador e a resolução de tal problema.
Outro ponto importante, como trabalho futuro, é a classificação de intensidade. Na implementação apresentada, apenas a classificação da Valência foi considerada.


## Referências

[1] NLM. Faceli K, Lorena AC, Gama J, Carvalho ACP de LF de. **Inteligência artificial: uma abordagem de aprendizado de máquina**. Rio de Janeiro: LTC, 2011.

[2] [Aprendizado de Máquina com Python](https://iascblog.wordpress.com/2017/03/17/aprendizado-de-maquina-supervisionado-com-python/)

[3] [Código de Exemplo](https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/)

[4] [Cross Validation](http://juliocesarbatista.com/post/Cross-validation-testando-o-desempenho-de-um-classificador/)

[5] [Scikit-learn - Trabalhando com Textos](http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)